# Installing Dependencies

In [1]:
import numpy as np

# BLAST Algorithm With E-Coli Data (W-Mer = 12)

Local alignment, aligning different regions of sequences that are seen to have similar sequence motifs, can be done through the BLAST algorithm.

The BLAST algorithm exploits the property of database searching where most of the target sequences that are found will often be unrelated to the query sequence. For example, if we are looking for different sequences of length 100 and want to reject matches that are less than 90% identical, then we do not need to look for sequences not containing a consecutive stretch of ten matching nucleotides in a row (often called the pigeonhole principle).

The BLAST algorithm does local alignment of a sequence of length m in a database of length n in O(m), through these five steps:
- Split query into overlapping words of length W (W-mers)
- Find “neighborhood” of similar words for each word
- Lookup each word in the neighborhood in the hash table to find the location in the database where each word occurs.
- Extend the seeds until the score of the alignment drops below some threshold.
- Report matches with overall highest scores

Blast Algorithm Pictoral Representation:
![blast.png](blast.png)

Article explaining this in more depth: https://medium.com/analytics-vidhya/matching-genetic-sequences-through-the-blast-and-karp-rabin-algorithm-ffebc810a9d0

In [10]:
dna = open("e_coli_seq.txt") # open arbitrary list of e.coli sequences
query = open("query.txt") # one-line query --> can construct 12-mers off of this

gene = []
query_seq = []
words_seq = []
hsp_word = []

# Gene represents the sets of nucleotides, arranged in the order of the genetic sequences in the text file
for line in dna:
    if line[0] != '>':
        end = len(line)
        for words in line[0:end-1]:
            gene.append(words) 

# Query represents the sets of nucleotides of the query sequence, arranged in the order seen in the text file
for line in query:
    if line[0] != '>':
        end = len(line)
        for words in line[0:end]:
            query_seq.append(words)

In [11]:
# Building a list of W-mers off of the query-sequence
for i in range(0,len(query_seq)):
    if i+11	< len(query_seq):
        words_seq.append(query_seq[i:i+12])

In [23]:
hsp_word = []
def high_scoring_pairs(): # finding the high scoring pairs in the data
    score = 0
    for i in range(0,len(words_seq)):
        k, score_init, index = 0, 0, 0 # finding a different score for every W-mer of the query sequence
        while(len(gene) >= k+12): 
            score = 0
            for j in range(k,k+12):
                if words_seq[i][j-k] == gene[j]: # adds to the score if the query sequence matches the nucleotide seq
                    score += 5
                if words_seq[i][j-k] != gene[j]: # removes from the score if the query sequence doesn't match
                    score -= 4
            if score > score_init: # checks if this was the highest score that was seen and then changes it
                score_init = score
                index = k
            k = k+1
        hsp_word.append([score_init,i,index]) # finds all the highest scores to find most optimal W-mers
    print(hsp_word)
high_scoring_pairs()

[[24, 0, 1479], [24, 1, 573], [24, 2, 574], [33, 3, 575], [33, 4, 576], [42, 5, 577], [33, 6, 167], [33, 7, 168], [33, 8, 626], [33, 9, 627], [33, 10, 628], [33, 11, 563], [24, 12, 70], [33, 13, 616], [42, 14, 1656], [42, 15, 1657], [33, 16, 535], [33, 17, 1659], [33, 18, 1660], [33, 19, 214], [33, 20, 922], [33, 21, 205], [33, 22, 399], [33, 23, 1915], [33, 24, 401], [24, 25, 209], [33, 26, 2056], [33, 27, 2696], [42, 28, 2697], [51, 29, 2698], [60, 30, 2699], [60, 31, 2700], [51, 32, 2701], [42, 33, 2702], [33, 34, 2597], [33, 35, 1453], [33, 36, 1454], [33, 37, 255], [33, 38, 903], [33, 39, 2708], [33, 40, 81], [42, 41, 82], [42, 42, 83], [42, 43, 387], [42, 44, 388], [51, 45, 2714], [60, 46, 2715], [60, 47, 2716], [60, 48, 2717], [60, 49, 2718], [60, 50, 2719], [60, 51, 2720]]


In [33]:
arr = np.array(hsp_word,dtype = int)
thres_max = []
ref = arr.max(axis=0)[0]

def threshold(): # similar process to what is seen above with hsp()
    for i in range(0,len(hsp_word)):
        cur_score = 0
        if arr[i][0] == ref:
            up = int(arr[i][1])
            up_left = len(query_seq)-up-12
            for j in range(up,-1,-1):
                if query_seq[j] == gene[int(int(arr[i][2])+ j - up)]:
                    cur_score += 5
                if query_seq[j] != gene[int(int(arr[i][2])+ j - up)] :
                    cur_score -= 4
            for k in range(up+12,len(query_seq)) :
                if query_seq[k] == gene[int((int(arr[i][2])+12)+k - up - 12)] :
                    cur_score += 5
                if query_seq[k] != gene[int((int(arr[i][2])+12)+k - up - 12)] :
                    cur_score -= 4
            thres_max.append([cur_score,i])
threshold()

# Displaying the final results
def dispx():
    for i in range(0,len(thres_max)):
        print("Query:")
        print(''.join(map(str,query_seq[0:len(query_seq)-1])))
        print("Sequence:")
        print(gene[int(hsp_word[int(thres_max[i][1])][2]) : int(hsp_word[int(thres_max[i][1])][2])+12])
        print()
dispx()

Query:
CGTGAGTCAGCTATTGAACTGGCCGCGCAATGGAAGAGTTGTTAATCCGCAAAATCTGGCAA
Sequence:
['T', 'G', 'G', 'A', 'A', 'G', 'A', 'G', 'T', 'T', 'G', 'T']

Query:
CGTGAGTCAGCTATTGAACTGGCCGCGCAATGGAAGAGTTGTTAATCCGCAAAATCTGGCAA
Sequence:
['G', 'G', 'A', 'A', 'G', 'A', 'G', 'T', 'T', 'G', 'T', 'T']

Query:
CGTGAGTCAGCTATTGAACTGGCCGCGCAATGGAAGAGTTGTTAATCCGCAAAATCTGGCAA
Sequence:
['C', 'C', 'G', 'C', 'A', 'A', 'A', 'A', 'T', 'C', 'T', 'G']

Query:
CGTGAGTCAGCTATTGAACTGGCCGCGCAATGGAAGAGTTGTTAATCCGCAAAATCTGGCAA
Sequence:
['C', 'G', 'C', 'A', 'A', 'A', 'A', 'T', 'C', 'T', 'G', 'G']

Query:
CGTGAGTCAGCTATTGAACTGGCCGCGCAATGGAAGAGTTGTTAATCCGCAAAATCTGGCAA
Sequence:
['G', 'C', 'A', 'A', 'A', 'A', 'T', 'C', 'T', 'G', 'G', 'C']

Query:
CGTGAGTCAGCTATTGAACTGGCCGCGCAATGGAAGAGTTGTTAATCCGCAAAATCTGGCAA
Sequence:
['C', 'A', 'A', 'A', 'A', 'T', 'C', 'T', 'G', 'G', 'C', 'A']

Query:
CGTGAGTCAGCTATTGAACTGGCCGCGCAATGGAAGAGTTGTTAATCCGCAAAATCTGGCAA
Sequence:
['A', 'A', 'A', 'A', 'T', 'C', 'T', 'G', 'G', 'C', 'A', 'A']

Query: